[View in Colaboratory](https://colab.research.google.com/github/ankanbhunia/Colab/blob/master/DLSETUP_.ipynb)

In [3]:
#@title Run GPU Machines { vertical-output: true }
Initialize = True #@param {type:"boolean"}
Install_Packages = True #@param {type:"boolean"}
Check_GPU = False #@param {type:"boolean"}
Remove_Trashes = False #@param {type:"boolean"}
Get_Jupyter_URL = False #@param {type:"boolean"}
Server = "ngrok" #@param ["localtunnel", "ngrok"]
Domain = "" #@param {type:"string"}
if Initialize:
  get_ipython().system_raw('git clone https://github.com/ankanbhunia/Colab.git Colab; cp Colab/* . ;rm -r Colab')
  exec(open('Colabstart.py').read())
if Install_Packages:
  load()
if Check_GPU:
  import tensorflow as tf
  print ('Available GPU : '+tf.test.gpu_device_name())
if Remove_Trashes:
  get_ipython().system_raw('python trash.py &')
if Get_Jupyter_URL:
  if Server == 'localtunnel':
    localtunnel(Domain)
  elif Server == 'ngrok':
    ngrok()


100%|██████████| 24/24 [01:13<00:00,  3.08s/it]


In [0]:
#@title Unzip dataset
path = "Cycle/horse2zebra.zip" #@param {type:"string"}
import os
if os.path.isfile(path):
  if path.endswith('zip'):
    get_ipython().system_raw('unzip '+ path)

  if path.endswith('rar'):
    get_ipython().system_raw('unrar x  '+ path)


In [7]:
Show_GPU_usage = True #@param {type:"boolean"}
Show_Running_Programs = True #@param {type:"boolean"}

if Show_GPU_usage:
  try:
    print ('GPU Usage : '+ get_ipython().getoutput('gpustat')[1])
  except:
    print ('No GPU')
print('')
if Show_Running_Programs:
  print ('Running Programs : ')
  print(task())

GPU Usage : [0] Tesla K80        | 43'C,  93 % | 11033 / 11439 MB |

Running Programs : 
    PID   memory      time           process
1    89    56616  00:00:02  /usr/bin/python2
2    97   144300  00:00:05  /usr/bin/python3
3  1603   739204  00:00:22  /usr/bin/python2
4  3079  2043464  00:22:14  /usr/bin/python3


In [0]:
#@title Kill Process
PID = "55" #@param {type:"string"}
get_ipython().getoutput('kill '+str(PID))

['/bin/bash: line 0: kill: (55) - No such process']

In [0]:

Reset_All = True #@param {type:"boolean"}
if Reset_All:
  get_ipython().getoutput('kill -9 -1')

In [0]:


def _activation_summary(x):
  
    tensor_name = re.sub('%s_[0-9]*/' % 'mp', '', x.op.name)
    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))


def _variable_on_cpu(name, shape, initializer):

    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer)

    return var


def _variable_with_weight_decay(name, shape, stddev, wd):

    var = _variable_on_cpu(name, shape, tf.random_normal_initializer(stddev=stddev))
    if wd:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)

    return var
  
class BatchNorm(object):

    def __init__(self, epsilon=1e-5, momentum=0.9, name="batch_norm"):

      with tf.variable_scope(name):
          self.epsilon = epsilon
          self.momentum = momentum
          self.name = name

    def __call__(self, x):

        return tf.contrib.layers.batch_norm(x, decay=self.momentum, updates_collections=None,
                                            epsilon=self.epsilon, scale=True, scope=self.name)


In [0]:

def multi_scale_block(in_con, in_dim, out_dim, is_bn=False):

    with tf.variable_scope('con_9') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[9, 9, in_dim, out_dim],
                                             stddev=0.01, wd=0.0005)
        con_9 = tf.nn.conv2d(in_con, kernel, [1, 1, 1, 1], padding='SAME', name=scope.name)
        _activation_summary(con_9)

    with tf.variable_scope('con_7') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[7, 7, in_dim, out_dim],
                                             stddev=0.01, wd=0.0005)
        con_7 = tf.nn.conv2d(in_con, kernel, [1, 1, 1, 1], padding='SAME', name=scope.name)
        _activation_summary(con_7)

    with tf.variable_scope('con_5') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5, in_dim, out_dim],
                                             stddev=0.01, wd=0.0005)
        con_5 = tf.nn.conv2d(in_con, kernel, [1, 1, 1, 1], padding='SAME', name=scope.name)
        _activation_summary(con_5)

    with tf.variable_scope('con_3') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[3, 3, in_dim, out_dim],
                                             stddev=0.01, wd=0.0005)
        con_3 = tf.nn.conv2d(in_con, kernel, [1, 1, 1, 1], padding='SAME', name=scope.name)
        _activation_summary(con_3)

    with tf.variable_scope('concat') as scope:
        concat = tf.concat([con_9, con_7, con_5, con_3], 3, name=scope.name)
        biases = _variable_on_cpu('biases', [out_dim * 4], tf.constant_initializer(0))
        bias = tf.nn.bias_add(concat, biases)

        if is_bn:
            bn = BatchNorm()
            bias = bn(bias)

        msb = tf.nn.relu(bias)
        _activation_summary(msb)

    return msb



In [0]:
def inference_bn(images):
    
    with tf.variable_scope('con1') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[9, 9, 3, 64],
                                             stddev=0.01, wd=0.0005)
        con = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME', name=scope.name)
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0))
        bias = tf.nn.bias_add(con, biases)
        con1 = tf.nn.relu(bias)
        _activation_summary(con1)

    # msb_con2
    with tf.variable_scope('msb_con2'):
        msb_con2 = multi_scale_block(con1, 64, 16, is_bn=True)

    # pool_msb_con2
    with tf.variable_scope('pool_msb_con2') as scope:
        pool_msb_con2 = tf.nn.max_pool(msb_con2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME',
                                       name=scope.name)

    # msb_con3
    with tf.variable_scope('msb_con3'):
        msb_con3 = multi_scale_block(pool_msb_con2, 64, 32, is_bn=True)

    # msb_con4
    with tf.variable_scope('msb_con4'):
        msb_con4 = multi_scale_block(msb_con3, 128, 32, is_bn=True)

    # pool_msb_con4
    with tf.variable_scope('pool_msb_con4') as scope:
        pool_msb_con4 = tf.nn.max_pool(msb_con4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME',
                                       name=scope.name)

    # msb_con5
    with tf.variable_scope('msb_con5'):
        msb_con5 = multi_scale_block(pool_msb_con4, 128, 64, is_bn=True)

    # msb_con6
    with tf.variable_scope('msb_con6'):
        msb_con6 = multi_scale_block(msb_con5, 256, 64, is_bn=True)

    # mpl_con7
    with tf.variable_scope('mpl_con7') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[1, 1, 256, 1000], stddev=0.001, wd=0.0005)
        con = tf.nn.conv2d(msb_con6, kernel, [1, 1, 1, 1], padding='SAME', name=scope.name)
        biases = _variable_on_cpu('biases', [1000], tf.constant_initializer(0))
        bias = tf.nn.bias_add(con, biases)
        mpl_con7 = tf.nn.relu(bias)
        _activation_summary(mpl_con7)

    # con_out
    with tf.variable_scope('con_out') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[1, 1, 1000, 1], stddev=0.001, wd=0.0005)
        con = tf.nn.conv2d(mpl_con7, kernel, [1, 1, 1, 1], padding='SAME', name=scope.name)
        biases = _variable_on_cpu('biases', [1], tf.constant_initializer(0))
        bias = tf.nn.bias_add(con, biases)

        bn = BatchNorm()
        bias = bn(bias)

        con_out = tf.nn.relu(tf.nn.sigmoid(bias))
        _activation_summary(con_out)

    image_out = con_out

    tf.summary.image("con_img", image_out)

    return image_out